<a href="https://colab.research.google.com/github/anupammalakar/stock-trading/blob/main/YFINANCE_EXPERIMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

In [2]:
import yfinance as yahooFinance

# in order to specify start date and
# end date we need datetime package
import datetime
import pandas as pd
from pandas.io.html import Pattern

# Python3 code here creating class
class Zone:
    def __init__(self, stock_name,legin_date, legout_date, price, zone_type, time_frame, current_price, last_update_date):
        self.stock_name = stock_name
        self.legin_date = legin_date
        self.legout_date = legout_date
        self.price = price
        self.zone_type = zone_type
        self.time_frame = time_frame
        self.current_price = current_price
        self.last_update_date = last_update_date

class Candle:
    def __init__(self, open, high, low, close, date):
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.date = date

class Threshold:
    def __init__(self):
        self.legin_boring_wick_lessthan_threshold_perct = 0.5
        self.legout_boring_wick_lessthan_threshold_perct = 0.5
        self.range_perct = .25

In [3]:


def data_cleaning(data):
  # Creating Empty DataFrame and Storing it in variable df
  df = pd.DataFrame()
  df = pd.DataFrame(data)

  # Resetting the index date as date column and index as numbers
  if not ('Date' in df):
    df.reset_index(level=0, inplace=True)

  df = df[["Date","Open","High","Low","Close"]]
  df = df.sort_values(by='Date', ascending=False)
  df= df.rename(columns=str.lower)
  return df

def validating_filled_candle_pair(candle_prev,candle,threshold):
  legout_full = abs(candle.high - candle.low)
  legout_body = abs(candle.close - candle.open)
  legin_full = abs(candle_prev.high - candle_prev.low)
  legin_body = abs(candle_prev.close - candle_prev.open)
  #if(legin_full != 0):
  legin_wick_perct = (legin_full - legin_body) / legin_full
  legout_wick_perct = (legout_full - legout_body) / legout_full

  #Checking date of a pair of candles not exceeding min of 3 to 5 days
  dt = pd.to_datetime(candle_prev.date, format='%Y/%m/%d')
  dt1 = pd.to_datetime(candle.date, format='%Y/%m/%d')

  #print(candle.date,'  VS  ', candle_prev.date, ' => ',open,"  legin wick: ",legin_wick_perct, "  legout wick: ",legout_wick_perct)

  #print(getattr(row,'date'),'  VS  ', getattr(prev,'date'), ' => ',open)
  if(legin_wick_perct<threshold.legin_boring_wick_lessthan_threshold_perct and legout_wick_perct < threshold.legout_boring_wick_lessthan_threshold_perct and (dt1-dt).days < 7):
    return True
  else:
    return False


In [4]:
from datetime import datetime
def find_gap_up_daily(stock_name,time_frame,data):
  threshold = Threshold()
  df = data_cleaning(data)

  #legin_boring_wick_lessthan_threshold_perct = 0.5
  #legout_boring_wick_lessthan_threshold_perct = 0.5
  #range_perct = .30
  prev = df.iloc[0]
  current_price = prev.close
  last_update_date = prev.date
  price_range_high = current_price * (1 + threshold.range_perct)
  price_range_low = current_price * (1 - threshold.range_perct)
  threshhold_low = prev.low

  zones = []


  for i,row in enumerate(df.itertuples()):

      #Give a proper check in the prev and current candle
      low = getattr(prev,'low')
      low_prev = getattr(row,'low')
      high = getattr(prev,'high')
      high_prev = getattr(row,'high')
      pattern_low = min(low,low_prev)
      pattern_high = max(high,high_prev)

      open = getattr(prev,'open')
      close = getattr(prev,'close')
      open_prev = getattr(row,'open')
      close_prev = getattr(row,'close')

      date_prev = getattr(row,'date')
      date = getattr(prev,'date')


      if(pattern_low < price_range_low):
        break
      if(pattern_low > threshhold_low):
        continue

      if(open < close and open_prev < close_prev and open > close_prev and close_prev <= threshhold_low):
        candle_prev = Candle(open_prev,high_prev,low_prev,close_prev,date_prev)
        candle = Candle(open,high,low,close,date)
        if(validating_filled_candle_pair(candle_prev,candle,threshold)):
          zone = Zone(stock_name,getattr(row,'date'),getattr(prev,'date'),open,"GAP_UP_IN_SAME_DIR",time_frame,current_price,last_update_date)
          zones.append(zone)

      if(open < close and open_prev > close_prev and close_prev <= open and low_prev <= low and low <= threshhold_low):
        candle_prev = Candle(open_prev,high_prev,low_prev,close_prev,date_prev)
        candle = Candle(open,high,low,close,date)
        threshold.legout_boring_wick_lessthan_threshold_perct = .7
        if(validating_filled_candle_pair(candle_prev,candle,threshold)):
          zone = Zone(stock_name,getattr(row,'date'),getattr(prev,'date'),open,"PIVOT_DZ",time_frame,current_price,last_update_date)
          zones.append(zone)


      threshhold_low = min(low,low_prev)
      #threshhold_low = min(open_prev,close,close_prev)

      prev = row
  return zones

In [5]:
from google.colab import files
#stocks = ['HINDALCO.NS']

#"""
stocks = ['AARTIIND.NS','ABB.NS','ABBOTINDIA.NS','ABCAPITAL.NS','ABFRL.NS','ACC.NS','ADANIENT.NS','ADANIPORTS.NS','ALKEM.NS','AMBUJACEM.NS','APOLLOHOSP.NS','APOLLOTYRE.NS','ASHOKLEY.NS','ASIANPAINT.NS','ASTRAL.NS','ATUL.NS','AUBANK.NS','AUROPHARMA.NS','AXISBANK.NS','BAJAJ-AUTO.NS','BAJAJFINSV.NS',
'BAJFINANCE.NS','BALKRISIND.NS','BALRAMCHIN.NS','BANDHANBNK.NS','BANKBARODA.NS','BANKEX.NS','BANKNIFTY.NS','BATAINDIA.NS','BEL.NS','BERGEPAINT.NS','BHARATFORG.NS','BHARTIARTL.NS','BHEL.NS','BIOCON.NS','BOSCHLTD.NS','BPCL.NS','BRITANNIA.NS','BSOFT.NS','CANBK.NS','CANFINHOME.NS','CHAMBLFERT.NS',
'CHOLAFIN.NS','CIPLA.NS','COALINDIA.NS','COFORGE.NS','COLPAL.NS','CONCOR.NS','COROMANDEL.NS','CROMPTON.NS','CUB.NS','CUMMINSIND.NS','DABUR.NS','DALBHARAT.NS','DEEPAKNTR.NS','DELTACORP.NS','DIVISLAB.NS','DIXON.NS','DLF.NS','DRREDDY.NS','EICHERMOT.NS','ESCORTS.NS','EXIDEIND.NS','FEDERALBNK.NS',
'FINNIFTY.NS','GAIL.NS','GLENMARK.NS','GMRINFRA.NS','GNFC.NS','GODREJCP.NS','GODREJPROP.NS','GRANULES.NS','GRASIM.NS','GUJGASLTD.NS','HAL.NS','HAVELLS.NS','HCLTECH.NS','HDFCAMC.NS','HDFCBANK.NS','HDFCLIFE.NS','HEROMOTOCO.NS','HINDALCO.NS','HINDCOPPER.NS','HINDPETRO.NS','HINDUNILVR.NS','IBULHSGFIN.NS',
'ICICIBANK.NS','ICICIGI.NS','ICICIPRULI.NS','IDEA.NS','IDFC.NS','IDFCFIRSTB.NS','IEX.NS','IGL.NS','INDHOTEL.NS','INDIACEM.NS','INDIAMART.NS','INDIGO.NS','INDUSINDBK.NS','INDUSTOWER.NS','INFY.NS','INTELLECT.NS','IOC.NS','IPCALAB.NS','IRCTC.NS','ITC.NS','JINDALSTEL.NS','JKCEMENT.NS','JSWSTEEL.NS',
'JUBLFOOD.NS','KOTAKBANK.NS','L&TFH.NS','LALPATHLAB.NS','LAURUSLABS.NS','LICHSGFIN.NS','LT.NS','LTIM.NS','LTTS.NS','LUPIN.NS','M&M.NS','M&MFIN.NS','MANAPPURAM.NS','MARICO.NS','MARUTI.NS','MCDOWELL-N.NS','MCX.NS','METROPOLIS.NS','MFSL.NS','MGL.NS','MIDCPNIFTY.NS','MOTHERSON.NS','MPHASIS.NS','MRF.NS',
'MUTHOOTFIN.NS','NATIONALUM.NS','NAUKRI.NS','NAVINFLUOR.NS','NESTLEIND.NS','NIFTY.NS','NMDC.NS','NTPC.NS','OBEROIRLTY.NS','OFSS.NS','ONGC.NS','PAGEIND.NS','PEL.NS','PERSISTENT.NS','PETRONET.NS','PFC.NS','PIDILITIND.NS','PIIND.NS','PNB.NS','POLYCAB.NS','POWERGRID.NS','PVRINOX.NS','RAMCOCEM.NS','RBLBANK.NS',
'RECLTD.NS','RELIANCE.NS','SAIL.NS','SBICARD.NS','SBILIFE.NS','SBIN.NS','SENSEX.NS','SENSEX50.NS','SHREECEM.NS','SHRIRAMFIN.NS','SIEMENS.NS','SRF.NS','SUNPHARMA.NS','SUNTV.NS','SX40.NS','SYNGENE.NS','TATACHEM.NS','TATACOMM.NS','TATACONSUM.NS','TATAMOTORS.NS','TATAPOWER.NS','TATASTEEL.NS','TCS.NS',
'TECHM.NS','TITAN.NS','TORNTPHARM.NS','TRENT.NS','TVSMOTOR.NS','UBL.NS','ULTRACEMCO.NS','UPL.NS','VEDL.NS','VOLTAS.NS','WIPRO.NS','ZEEL.NS','ZYDUSLIFE.NS']
#"""

results_df = pd.DataFrame()

for stock in stocks:

  data = yahooFinance.download(stock,'2022-06-22','2023-07-27')
  if(data.empty):
    continue
  zones = find_gap_up_daily(stock,'1D',data)
  # fill dataframe with one row per object, one attribute per column
  df = pd.DataFrame([t.__dict__ for t in zones ])
  results_df = results_df.append(df)

  #break
  # Accessing object value using a for loop
  #for obj in zones:
      #print(obj.stock_name, obj.time_frame, obj.legin_date, obj.legout_date, obj.price,obj.zone_type, obj.current_price, obj.last_update_date)
      #print(obj.stock_name, obj.time_frame, obj.legin_date, obj.legout_date, obj.price,obj.zone_type)
results_df.to_csv('output.csv', encoding = 'utf-8-sig')
files.download('output.csv')

[*********************100%%**********************]  1 of 1 completed


<ipython-input-5-5400eb824a22>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(df)


[*********************100%%**********************]  1 of 1 completed


<ipython-input-5-5400eb824a22>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(df)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

...

<ipython-input-5-5400eb824a22>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(df)


[*********************100%%**********************]  1 of 1 completed


<ipython-input-5-5400eb824a22>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(df)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
results_df

,stock_name,legin_date,legout_date,price,zone_type,time_frame,current_price,last_update_date
0,AARTIIND.NS,2023-07-13,2023-07-14,450.799988,PIVOT_DZ,1D,464.200012,2023-07-26
0,ABB.NS,2023-06-01,2023-06-02,3977.000000,PIVOT_DZ,1D,4398.500000,2023-07-26
1,ABB.NS,2023-05-05,2023-05-08,3728.449951,GAP_UP_IN_SAME_DIR,1D,4398.500000,2023-07-26
2,ABB.NS,2023-05-04,2023-05-05,3653.000000,GAP_UP_IN_SAME_DIR,1D,4398.500000,2023-07-26
0,ABBOTINDIA.NS,2023-06-14,2023-06-15,22160.000000,PIVOT_DZ,1D,22945.449219,2023-07-26
...,...,...,...,...,...,...,...,...
0,UPL.NS,2023-07-25,2023-07-26,628.000000,PIVOT_DZ,1D,631.900024,2023-07-26
0,VOLTAS.NS,2023-01-27,2023-01-30,747.849976,PIVOT_DZ,1D,765.000000,2023-07-26
0,WIPRO.NS,2023-07-12,2023-07-13,391.850006,PIVOT_DZ,1D,402.750000,2023-07-26
1,WIPRO.NS,2023-04-21,2023-04-24,375.000000,GAP_UP_IN_SAME_DIR,1D,402.750000,2023-07-26
